<a href="https://colab.research.google.com/github/BarbaLoira/Analysis-Essays-NLP/blob/master/Analysis_of_essays_with_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Install uol-redacoes-xml

In [2]:
!pip install git+https://github.com/gpassero/uol-redacoes-xml.git

  Cloning https://github.com/gpassero/uol-redacoes-xml.git to /tmp/pip-req-build-wvsr5de_
  Stored in directory: /tmp/pip-ephem-wheel-cache-f4ynx0tt/wheels/1b/d6/69/d465fd205ecd2058fe620852fb31bdf5cd0b9697ec2d7548eb
Successfully built uol-redacoes-xml


#import and load uol_redacoes_xml

In [3]:
import nltk
nltk.download('punkt')
import uol_redacoes_xml

essays = uol_redacoes_xml.load()
# print(len(essays)) #~2000

# print(essays[0].text) #texto original da primeira redação

# print(  [attr for attr in essays[0].__dir__() if not attr.startswith('_')]   ) # exibe os atributos do objeto de redação (exceto os privados, que começam com '_')

 



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


UOL essays load warnings: 
No text  ->  3
Final score != from sum of criteria score  ->  45
Not 5 criteria  ->  7
Total warnings: 55


#Imports Useful

In [0]:
from sklearn.model_selection import cross_validate 
from sklearn.svm import SVC
from sklearn.cluster import KMeans
from sklearn import datasets

from uol_redacoes_xml.reader.commons import kfold_cross_validation

import numpy as np

import matplotlib
import matplotlib.pyplot as plt

import pandas as pd

import spacy
#!python -m spacy download pt
nlp = spacy.load('pt')


import pyphen
#!pip install pyphen
#dic = pyphen.Pyphen(lang='pt')
#dic.inserted('macaco')
pyphenPt = pyphen.Pyphen(lang='pt')

#!pip install textstat
from textstat.textstat import legacy_round 

 

#formatando o texto

In [0]:
variables  = ['title', 'text' , 'final_score' ]
data = [ [getattr(i,attr) for attr in essays[0].__dir__() if not attr.startswith('_') and not attr.startswith('prompt') and not attr.startswith('url') and not attr.startswith('fixed_text') and not attr.startswith('errors') and not attr.startswith('get_features') and not attr.startswith('comments') and not attr.startswith('criteria_scores') ] for i in essays]

text_formatted = pd.DataFrame(data ,columns = variables )
text_formatted
#text_formatted.head()
#text_formatted.tail()
#text_formatted["text"][0]

#Flesch Reading Ease

In [108]:


text = text_formatted["text"][2]
flesch_reading_ease('O Doutor Armando foi até a esquina esperá o filho que chegava da escola. Nisso, a Maria ficou em casa preparando o almoço.Quando eles chegarão em casa a Maria tava na cozinha preparando a famosa receita da família boa pra caramba o bolo de fubá cremoso.')

countWords = len(count_words_text(text))

countSentence = len(list(nlp(text_formatted["text"][2]).sents))
  

countSylable = len((pyphenPt.inserted(text_formatted["text"][2])).split("-"))


206.836 - ((1.105 * (countWords/countSentence)) - (84.6 * (countSylable / countWords)))




54.22

#Util

In [0]:
def break_sentences(text):   
    doc = nlp(text)
    return list(doc.sents) 
  
# Returns Number of Words in the text 
def word_count(text): 
    sentences = break_sentences(text) 
    words = 0
    for sentence in sentences: 
        words += len([token for token in sentence]) 
    return words 
  
# Returns the number of sentences in the text 
def sentence_count(text): 
    sentences = break_sentences(text) 
    return len(sentences)  
  
  
  # Returns average sentence length 
def avg_sentence_length(text): 
    words = word_count(text) 
    sentences = sentence_count(text) 
    average_sentence_length = float(words / sentences) 
    return average_sentence_length 
  
def syllables_count(word): 
    return len((pyphenPt.inserted(word)).split("-"))
  
  
  # Returns the average number of syllables per 
# word in the text 
def avg_syllables_per_word(text): 
    syllable = syllables_count(text) 
    words = word_count(text) 
    ASPW = float(syllable) / float(words) 
    return legacy_round(ASPW, 1) 
  
  
# Return total Difficult Words in a text 
def difficult_words(text): 
  
    # Find all words in the text 
    words = [] 
    sentences = break_sentences(text) 
    for sentence in sentences: 
        words += [str(token) for token in sentence] 
  
    # difficult words are those with syllables >= 2 
    # easy_word_set is provide by Textstat as  
    # a list of common words 
    diff_words_set = set() 
      
    for word in words: 
        syllable_count = syllables_count(word) 
        if word not in easy_word_set and syllable_count >= 2: 
            diff_words_set.add(word) 
  
    return len(diff_words_set) 
  
# A word is polysyllablic if it has more than 3 syllables 
# this functions returns the number of all such words  
# present in the text 
def poly_syllable_count(text): 
    count = 0
    words = [] 
    sentences = break_sentences(text) 
    for sentence in sentences: 
        words += [token for token in sentence] 
      
  
    for word in words: 
        syllable_count = syllables_count(word) 
        if syllable_count >= 3: 
            count += 1
    return count 
  
  # muito facil        75 a 100     (primeiros 4 anos escolares)
  # facil              50 a 75      (quita a oitava serie)
  # pouco dificil      25 a 50      (colegial e o nivel superior)
  # muito dificil   abaixo de 25    (texto academico) 
def flesch_reading_ease(text): 
    """ 
        Implements Flesch Formula: 
        Reading Ease score = 206.835 - (1.015 × ASL) - (84.6 × ASW) 
        Here, 
          ASL = average sentence length (number of words  
                divided by number of sentences) 
          ASW = average word length in syllables (number of syllables  
                divided by number of words) 
    """
    FRE = 206.835 - float(1.015 * avg_sentence_length(text)) - float(84.6 * avg_syllables_per_word(text)) 
    return legacy_round(FRE, 2)   
  